# Allen Cahn Equation

In [102]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time

torch.set_default_dtype(torch.float64)

In [103]:
def sample_points(N_pts):
    
    # interior points
    N_domain_tot = N_pts**2
    x_int_train = torch.rand(N_domain_tot, 1, requires_grad=True)
    y_int_train = torch.rand(N_domain_tot, 1, requires_grad=True)
    
    # boundary
    # bottom points
    x_bottom = torch.rand(N_pts+1, 1)
    y_bottom = torch.zeros(N_pts+1, 1)
    # top points
    x_top = torch.rand(N_pts+1, 1)
    y_top = torch.ones(N_pts+1,1)
    # concatenate
    x_bound_train = torch.concat((x_bottom, x_top))
    y_bound_train = torch.concat((y_bottom, y_top))
    # left points
    x_left = torch.zeros(N_pts+1,1)
    y_left = torch.rand(N_pts+1,1)
    # concat
    x_bound_train = torch.concat((x_bound_train, x_left))
    y_bound_train = torch.concat((y_bound_train, y_left))
    # right points
    x_right = torch.ones(N_pts+1,1)
    y_right = torch.rand(N_pts+1,1)
    # concat
    x_bound_train = torch.concat((x_bound_train, x_right))
    y_bound_train = torch.concat((y_bound_train, y_right))
    
    return x_int_train, y_int_train, x_bound_train, y_bound_train



def u_true(a, x, y):
    return torch.sin(2* torch.pi * a * x) * torch.sin(2 * torch.pi * a * y)

def f(a, x, y):
    
    u = u_true(a, x,y)
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    
    return u_xx + u_yy + u**3 - u

def g(a, x,y):
    return u_true(a, x,y)

## Case 1: a=1

In [104]:
def Jacobian(c):
    
    u = A_int @ c
    
    ############ G(c):
    value1 = delta_u @ c + u**3 - u - rhs_pde.detach().numpy().reshape(-1)
    value2 = A_bd @ c - rhs_bd.detach().numpy().reshape(-1)
    rhs = np.concatenate((value1, value2))
    
    ########### Jacobian
    u = u.reshape((u.shape[0],1))
    J1 = delta_u + 3 * u ** 2 * A_int - A_int
    J = np.row_stack((J1,A_bd))
    
    return J, rhs

In [105]:
# 
torch.manual_seed(5)
a = 1
# generate training samples
N_pts = 30
x_int_train, y_int_train, x_bound_train, y_bound_train = sample_points(N_pts)

# generate test samples
num_pts = 100
xx= np.linspace(0, 1, num_pts)
yy = np.linspace(0, 1, num_pts)
XX, YY = np.meshgrid(xx, yy)
XX_test, YY_test = XX.reshape(num_pts**2,1), YY.reshape(num_pts**2,1)
XX_test, YY_test = torch.from_numpy(XX_test), torch.from_numpy(YY_test)
XX, YY = torch.from_numpy(XX), torch.from_numpy(YY)
u_test = u_true(a,XX,YY)

u_test = u_test.detach().numpy()

# compute rhs
rhs_pde = f(a, x_int_train, y_int_train)
rhs_bd = g(a, x_bound_train, y_bound_train)

# generate random features
N = 200
sigma = 0.1
W = np.random.randn(2,N) / sigma
W_norm = np.linalg.norm(W,axis=0) ** 2
b = np.random.normal(0,2*np.pi,(N,))


x_train = torch.cat((x_int_train,y_int_train), dim=1)
x_train = x_train.detach().numpy()
A_int = np.cos(x_train @ W + b)

x_bd = torch.cat((x_bound_train,y_bound_train), dim=1)
x_bd = x_bd.detach().numpy()
A_bd = np.cos(x_bd @ W + b)

x_test = torch.cat((XX_test,YY_test), dim=1)
x_test = x_test.detach().numpy()
A_test = np.cos(x_test @ W + b)

# some matrices
delta_u = - A_int * W_norm

# Initial guess for parameters
c = np.zeros((N,))
#c = np.random.rand(N)
# Iter 100 times
Iter = 30
# test error
test_err = []
start = time.time()
for i in range(Iter):
    
    J, rhs = Jacobian(c)
    
    delta_c = np.linalg.pinv(J) @ (-rhs)
    c = c + delta_c

    ELM_pred = A_test @ c
    ELM_pred = ELM_pred.reshape(num_pts, num_pts)

    err_ELM = np.sum( (u_test - ELM_pred)**2 ) / np.size(u_test)
    test_err.append(err_ELM)
end = time.time()
print(f'Test error of RF model is {min(test_err):.2e}')
print(f'Training time is {end-start:.2f} seconds')


Test error of RF model is 2.55e-13
Training time is 0.66 seconds


In [32]:
# generate random features
N = 200
R = 5
W = np.random.uniform(-R,R,size=(2,N))
W_norm = np.linalg.norm(W,axis=0) ** 2

#x_train = torch.cat((x_int_train,y_int_train), dim=1)
#x_train = x_train.detach().numpy()
A_int = np.tanh(x_train @ W)

#x_bd = torch.cat((x_bound_train,y_bound_train), dim=1)
#x_bd = x_bd.detach().numpy()
A_bd = np.tanh(x_bd @ W )

#x_test = torch.cat((XX_test,YY_test), dim=1)
#x_test = x_test.detach().numpy()
A_test = np.tanh(x_test @ W)

# some matrices
delta_u = - 2 * A_int * (1-A_int**2) * W_norm

# Initial guess for parameters
c = np.zeros((N,))
#c = np.random.rand(N)
# Iter 100 times
Iter = 30
# test error
test_err = []
start = time.time()
for i in range(Iter):
    
    J, rhs = Jacobian(c)
    
    delta_c = np.linalg.pinv(J) @ (-rhs)
    c = c + delta_c

    ELM_pred = A_test @ c
    ELM_pred = ELM_pred.reshape(num_pts, num_pts)

    err_ELM = np.sum( (u_test - ELM_pred)**2 ) / np.size(u_test)
    test_err.append(err_ELM)
end = time.time()
print(f'Test error of RF model is {min(test_err):.2e}')
print(f'Training time is {end-start:.2f} seconds')


Test error of RF model is 6.16e-03
Training time is 0.64 seconds


## Case 2: a=10

In [171]:
# 
torch.manual_seed(5)
a = 10
# generate training samples
N_pts = 30
x_int_train, y_int_train, x_bound_train, y_bound_train = sample_points(N_pts)

# generate test samples
num_pts = 100
xx= np.linspace(0, 1, num_pts)
yy = np.linspace(0, 1, num_pts)
XX, YY = np.meshgrid(xx, yy)
XX_test, YY_test = XX.reshape(num_pts**2,1), YY.reshape(num_pts**2,1)
XX_test, YY_test = torch.from_numpy(XX_test), torch.from_numpy(YY_test)
XX, YY = torch.from_numpy(XX), torch.from_numpy(YY)
u_test = u_true(a,XX,YY)

u_test = u_test.detach().numpy()

# compute rhs
rhs_pde = f(a, x_int_train, y_int_train)
rhs_bd = g(a, x_bound_train, y_bound_train)

# generate random features
N = 200
sigma = 0.001
W = np.random.randn(2,N) / sigma
W_norm = np.linalg.norm(W,axis=0) ** 2
b = np.random.normal(0,2*np.pi,(N,))


x_train = torch.cat((x_int_train,y_int_train), dim=1)
x_train = x_train.detach().numpy()
A_int = np.cos(x_train @ W + b)

x_bd = torch.cat((x_bound_train,y_bound_train), dim=1)
x_bd = x_bd.detach().numpy()
A_bd = np.cos(x_bd @ W + b)

x_test = torch.cat((XX_test,YY_test), dim=1)
x_test = x_test.detach().numpy()
A_test = np.cos(x_test @ W + b)

# some matrices
delta_u = - A_int * W_norm

# Initial guess for parameters
c = np.zeros((N,))
#c = np.random.rand(N)
# Iter 100 times
Iter = 20
# test error
test_err = []
start = time.time()
for i in range(Iter):
    
    J, rhs = Jacobian(c)
    
    delta_c = np.linalg.pinv(J) @ (-rhs)
    c = c + delta_c

    ELM_pred = A_test @ c
    ELM_pred = ELM_pred.reshape(num_pts, num_pts)

    err_ELM = np.sum( (u_test - ELM_pred)**2 ) / np.size(u_test)
    test_err.append(err_ELM)
end = time.time()
print(f'Test error of RF model is {min(test_err):.2e}')
print(f'Training time is {end-start:.2f} seconds')


Test error of RF model is 2.45e-01
Training time is 0.44 seconds


In [83]:
# generate random features
N = 400
R = 1
W = np.random.uniform(-R,R,size=(2,N))
W_norm = np.linalg.norm(W,axis=0) ** 2

#x_train = torch.cat((x_int_train,y_int_train), dim=1)
#x_train = x_train.detach().numpy()
A_int = np.tanh(x_train @ W)

#x_bd = torch.cat((x_bound_train,y_bound_train), dim=1)
#x_bd = x_bd.detach().numpy()
A_bd = np.tanh(x_bd @ W )

#x_test = torch.cat((XX_test,YY_test), dim=1)
#x_test = x_test.detach().numpy()
A_test = np.tanh(x_test @ W)

# some matrices
delta_u = - 2 * A_int * (1-A_int**2) * W_norm

# Initial guess for parameters
c = np.zeros((N,))
#c = np.random.rand(N)
# Iter 100 times
Iter = 30
# test error
test_err = []
start = time.time()
for i in range(Iter):
    
    J, rhs = Jacobian(c)
    
    delta_c = np.linalg.pinv(J) @ (-rhs)
    c = c + delta_c

    ELM_pred = A_test @ c
    ELM_pred = ELM_pred.reshape(num_pts, num_pts)

    err_ELM = np.sum( (u_test - ELM_pred)**2 ) / np.size(u_test)
    test_err.append(err_ELM)
end = time.time()
print(f'Test error of RF model is {min(test_err):.2e}')
print(f'Training time is {end-start:.2f} seconds')


Test error of RF model is 1.82e+02
Training time is 1.34 seconds
